In [1]:
from collections import defaultdict
import os
import csv

from xml.etree.ElementTree import parse, Element, SubElement, ElementTree
import xml.etree.ElementTree as ET

In [ ]:

save_root2 = "xmls"

if not os.path.exists(save_root2):
    os.mkdir(save_root2)

def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

def write_xml(folder, filename, bbox_list):
    root = Element('annotation')
    SubElement(root, 'folder').text = folder
    SubElement(root, 'filename').text = filename
    SubElement(root, 'path').text = 'C:/Users/User/Documents/EBTECH/dataset/palmoiltree/raw/pic/' +  filename
    source = SubElement(root, 'source')
    SubElement(source, 'database').text = 'Unknown'
    indent(source)


    # Details from first entry
    e_filename, e_width, e_height, e_class_name, e_xmin, e_ymin, e_xmax, e_ymax = bbox_list[0]
    
    size = SubElement(root, 'size')
    SubElement(size, 'width').text = e_width
    SubElement(size, 'height').text = e_height
    SubElement(size, 'depth').text = '3'
    indent(size)

    SubElement(root, 'segmented').text = '0'

    for entry in bbox_list:
        e_filename, e_width, e_height, e_class_name, e_xmin, e_ymin, e_xmax, e_ymax = entry
        
        obj = SubElement(root, 'object')
        SubElement(obj, 'name').text = e_class_name
        SubElement(obj, 'pose').text = 'Unspecified'
        SubElement(obj, 'truncated').text = '0'
        SubElement(obj, 'difficult').text = '0'
        indent(obj)

        bbox = SubElement(obj, 'bndbox')
        SubElement(bbox, 'xmin').text = e_xmin
        SubElement(bbox, 'ymin').text = e_ymin
        SubElement(bbox, 'xmax').text = e_xmax
        SubElement(bbox, 'ymax').text = e_ymax
        indent(bbox)

    indent(root)
    tree = ElementTree(root)
    
    xml_filename = os.path.join('.', folder, os.path.splitext(filename)[0] + '.xml')
    tree.write(xml_filename)
    
entries_by_filename = defaultdict(list)

with open("C:/Users/User/Documents/EBTECH/dataset/palmoiltree/raw/csv/raw.csv", 'r', encoding='utf-8') as f_input_csv:
    csv_input = csv.reader(f_input_csv)
    header = next(csv_input)

    for row in csv_input:
        filename, width, height, class_name, xmin, ymin, xmax, ymax = row

        if class_name == "palm oil":
            entries_by_filename[filename].append(row)

for filename, entries in entries_by_filename.items():
    print(filename, len(entries))
    write_xml(save_root2, filename, entries)


In [2]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [5]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"unripe": 0,
                            "underripe": 1,
                            "ripe": 2,
                            "overripe": 3}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join('C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels', info_dict["filename"].split(".")[0]+".txt") 
    
    # Save the annotation to disk
    print(save_file_name)
    with open(save_file_name, 'w') as f:
        for line in print_buffer:
            f.write(line)
            f.write('\n')

In [6]:
# Get the annotations
from tqdm import tqdm
from collections import defaultdict
import os
import csv

from xml.etree.ElementTree import parse, Element, SubElement, ElementTree
import xml.etree.ElementTree as ET
annotations = [os.path.join('C:/Users/User/Documents/EBTECH/dataset/ffb/test', x) for x in os.listdir('C:/Users/User/Documents/EBTECH/dataset/ffb/test') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
# annotations = [os.path.join('C:/Users/User/Documents/EBTECH/dataset/palmoiltree/all', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

  7%|▋         | 25/340 [00:00<00:01, 237.64it/s]

C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_340.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_342.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_343.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_344.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_345.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_346.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_363.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_436.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_451.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_465.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_466.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\ActiOn_467.txt
C:/Users/User/Documents/EBTECH/dataset/f

 14%|█▍        | 49/340 [00:00<00:02, 131.26it/s]

C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1294.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1295.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1296.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1297.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1298.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1299.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1300.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1301.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1302.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1303.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1304.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1305.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels

 19%|█▉        | 65/340 [00:00<00:02, 119.82it/s]

...

 89%|████████▉ | 304/340 [00:03<00:00, 73.01it/s]


C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1561.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1562.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1563.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1564.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1565.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1566.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1567.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1568.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1569.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1570.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1571.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1572.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/label

 97%|█████████▋| 330/340 [00:03<00:00, 93.23it/s]


C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1579.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1580.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1581.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1582.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1583.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1584.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1585.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1586.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1587.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1588.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1589.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/labels\img_1590.txt
C:/Users/User/Documents/EBTECH/dataset/ffb/ripeness/valid/label

100%|██████████| 340/340 [00:03<00:00, 99.34it/s]